**2\. Анализ данных (Data Understanding)**

Цель шага – понять слабые и сильные стороны предоставленных данных, определить их достаточность, предложить идеи, как  
их использовать, и лучше понять процессы заказчика. Для этого мы строим графики, делаем выборки и рассчитываем  
статистики.

2.1 Сбор данных (Data collection)  
Выгружаем необходимые данные (или срез данных если их объем слишком велик) из источников.  
Версионируем данные средствами DVC.

2.2 Исследование данных (Data exploration)  
Исследуем данные, чтобы сформулировать гипотезы относительно того, как эти данные помогут решить задачу. Проверяем  
качество данных.

Ориентировочный список для проверки данных:

1. Загрузить репрезентативную выборку из набора данных  
2. Провести предварительный анализ всей выборки.  
* определить тип данных в каждом столбце  
* Категориальные данные (Номинальные, Порядковые)  
* Числовые данные (дискретные, непрерывные, интервальные, отношения)  
* При необходимости преобразовать данные к нужным типам  
* Проверить на выбросы, отсутствующие значения, невалидные значения(например в системе случился сбой и в поле с именемпопала длина просмотра).

(по результатам предварительного анализа сделать визуализацию, обычно это табличка с характеристиками или какой то из  
профайлеров)

3. На основе предыдущего анализа выполнить очистку данных (обработать выбросы, отсутствующие значения, удалить невалидные значения)  
4. Удалить из рассмотрения неинформативные данные. (лишние идентификаторы, служебные поля, поля с очень малым количеством значений)  
5. Провести статистический анализ оставшихся данных  
   1. рассчитать ключевые статистики для каждого типа данных  
   2. построить распределения (тип графика выбрать в зависимости от данных, часто полезно построить гистограмму, но иногда лучше воспользоваться линейным графиком или посмотреть распределение во времени с помощью scaterplot)  
6. Провести корреляционный анализ  
   1. Для количественных данных нормализовать данные и построить матрицу корреляции Пирсона  
   2. сделать выводы на основе матрицы (найти утечки данных, найти важные признаки линейно влияющие на целевой показатель, определить гипотезы по конструированию признаков)  
   3. Для количественных и порядковых данных \- построить матрицу корреляции Спирмена  сделать выводы аналогично предыдущему анализу, только учесть тип данных  
   4. Для всех данных построить матрицу корреляции Пфика  
   5. Сделать выводы на основе анализа, сделать оценку между всеми типами корреляции(на пересекающихся данных),  
   6. попытаться найти объяснения различиям.  
   7. Сделать выводы о наличии или отсутствии нелинейных связей.  
        
7.  Провести обработку данных, на основе выводов полученных в прошлых шагах.  
   1. провести дополнительную очистку  
   2. выполнить нужный тип энкодинга(если требуется)  
   3. Сконструировать новые признаки.

8. Построить Графики взаимодействия полученных данных с целевым показателем  
   1. Для количественных данных линейные графики на нормализованных данных  
   2. для категориальных данных с малым количеством категорий построить ScaterPlot во времени  
   3. для категориальных данных с большим количеством показателей построить heatmap во времени(обычно строят, только для ризнаков которые показывают высокие коэффициенты корреляции и потенциально интересны, пример такого графика спектрограмма в анализе звука)  
9. Сделать выводы на основе проведенного анализа и учитывая особенности планируемой архитектуры модели.  
   1. Какие данные и почему нельзя использовать в модели  
   2. какие данные можно использовать без преобразования  
   3. какие данные можно использовать выполнив преобразование  
   4. какие новые признаки нужно использовать и почему  
   5. есть ли смысл использовать один набор признаков или построить разные модели на подмножестве признаков и почему.  
   6. Выделить итоговый список необходимых данных  
10. Описать ожидания от модели на проанализированных данных

In [7]:
from lakefs_client import Configuration
from dotenv import load_dotenv
import os

load_dotenv()  # Загружает переменные из .env

def get_lakefs_config():
    config = Configuration()
    config.host = os.getenv("LAKEFS_HOST")
    config.username = os.getenv("LAKEFS_ACCESS_KEY")
    config.password = os.getenv("LAKEFS_SECRET_KEY")
    return config


In [11]:
from lakefs_client import Configuration
from lakefs_client import ApiClient
from lakefs_client.api import objects_api

def load_dataframe(configuration: Configuration, repository: str, ref: str, path: str):
    # Создаем клиента
    with ApiClient(configuration) as api_client:
        obj_api = objects_api.ObjectsApi(api_client)

        # Получаем объект
        #repository = "mfdp-fin-fraud-detection-data"
        #ref = "main"  # или другой commit/branch/tag
        #path = "ieee-fraud-detection/sample_submission.csv"

        response = obj_api.get_object(repository, ref, path)
        content = response.read()

        # Например, если CSV
        import pandas as pd
        from io import StringIO
        df = pd.read_csv(StringIO(content.decode('utf-8')))
        return df

In [15]:
# Получаем объект
configuration = get_lakefs_config()
repository = "mfdp-fin-fraud-detection-data"
ref = "main"  # или другой commit/branch/tag
path = "ieee-fraud-detection/sample_submission.csv"

df = load_dataframe(configuration, repository, ref, path)
df.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5
